# Reading the Data

In [1]:
from Bio import SeqIO
import numpy as np
import csv
from shared import *
from project_nader import *
from time import time

In [2]:
def read_reads():
    """
    Reads all the RNA reads from reads.fa and returns a list of sequences as strings.

    :return: a list of RNA sequence reads as strings
    """
    max_id_len = 600
    reads = np.zeros((1575, 2), dtype='U{:d}'.format(max_id_len))
    i = 0
    for seq_record in SeqIO.parse("reads.fa", "fasta"):
        reads[i, 0] = seq_record.id
        reads[i, 1] = str(seq_record.seq)
        i += 1
    return reads

In [3]:
def read_genome():
    """
    Reads the genome sequence from genome.fa and return the genome sequence as a string.

    :return: a string of the genome sequence
    """
    genome = None
    for seq_record in SeqIO.parse("genome.fa", "fasta"):
        genome = str(seq_record.seq)
    return genome

In [4]:
def read_known_genes():
    """
    Reads the un/known genes, isoforms, and exons from genes.tab and constructs objects for each
    and return the list of constructed genes.

    :return: known_genes (a list of known Gene objects), unknown_genes (a list of unknown Gene objects)
    """
    known_genes, known_isoforms, known_exons = {}, {}, {}
    unknown_genes, unknown_isoforms, unknown_exons = {}, {}, {}
    with open("genes.tab") as tsv:
        for line in csv.reader(tsv, dialect="excel-tab"):
            name = line[0]
            if name == 'gene':
                known_genes[line[1]] = line[2].split(';')
            elif name == 'isoform':
                known_isoforms[line[1]] = line[2].split(';')
            elif name == 'exon':
                known_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
            elif name == 'unknown_gene':
                unknown_genes[line[1]] = line[2].split(';')
            elif name == 'unknown_isoform':
                unknown_isoforms[line[1]] = line[2].split(';')
            elif name == 'unknown_exon':
                unknown_exons[line[1]] = Exon(line[1], int(line[2]), int(line[3]))
    # Create the known Isoform objects
    for k in known_isoforms:
        known_isoforms[k] = Isoform(k, [known_exons[key] for key in known_isoforms[k]])
        
    # Create the UNknown Isoform objects
    for k in unknown_isoforms:
        unknown_isoforms[k] = Isoform(k, [unknown_exons[key] for key in unknown_isoforms[k]])

    # Create the known Genes objects
    for k in known_genes:
        known_genes[k] = Gene(k, [known_isoforms[key] for key in known_genes[k]])
        
    # Create the UNknown Genes objects
    for k in unknown_genes:
        unknown_genes[k] = Gene(k, [unknown_isoforms[key] for key in unknown_genes[k]])
    return known_genes, unknown_genes

# Alignment Algorithm Class

In [5]:
MIN_INTRON_SIZE = 20
MAX_INTRON_SIZE = 10000


class Aligner:
    def __init__(self, genome_sequence, known_genes):
        """
        Initializes the aligner. Do all time intensive set up here. i.e. build suffix array.

        genome_sequence: a string (NOT TERMINATED BY '$') representing the bases of the of the genome
        known_genes: a python set of Gene objects (see shared.py) that represent known genes. You can get the isoforms
                     and exons from a Gene object

        Time limit: 500 seconds maximum on the provided data. Note that our server is probably faster than your machine,
                    so don't stress if you are close. Server is 1.25 times faster than the i7 CPU on my computer
        """
        self.sa = get_suffix_array(genome_sequence)
        L = get_bwt(genome_sequence, self.sa)
        self.occ, self.M = get_occ(L), get_M(get_F(L))
        self.known_genes = known_genes

    def align(self, read_sequence):
        """
        Returns an alignment to the genome sequence. An alignment is a list of pieces.
        Each piece consists of a start index in the read, a start index in the genome, and a length
        indicating how many bases are aligned in this piece. Note that mismatches are count as "aligned".

        Note that <read_start_2> >= <read_start_1> + <length_1>. If your algorithm produces an alignment that
        violates this, we will remove pieces from your alignment arbitrarily until consecutive pieces
        satisfy <read_start_2> >= <read_start_1> + <length_1>

        Return value must be in the form (also see the project pdf):
        [(<read_start_1>, <reference_start_1, length_1), (<read_start_2>, <reference_start_2, length_2), ...]

        If no good matches are found: return the best match you can find or return []

        Time limit: 0.5 seconds per read on average on the provided data.
        """
        # for gene in self.known_genes:

        pass

# Testing

In [6]:
# Main
reads = read_reads()
genome_sequence = read_genome()
known_genes, unknown_genes = read_known_genes()

In [7]:
print("length of the genome sequence: " + str(len(genome_sequence) + 1))

t = -time()
"""
aligner = Aligner(genome_sequence, known_genes)
t += time()
"""
# Initialization
sa = get_suffix_array(genome_sequence + '$')
L = get_bwt(genome_sequence, sa)
occ, M = get_occ(L), get_M(get_F(L))
# self.known_genes = known_genes

print("time to run Aligner.__init__: " + str(t + time()))

length of the genome sequence: 10949001
time to run Aligner.__init__: 314.40626215934753


In [39]:
t = -time()
known_transcriptome = {}
for gene in known_genes:
    known_transcriptome[gene] = {}
    for iso in known_genes[gene].isoforms:
        isoform, exons, total_len = '', [], 0
        for ex in iso.exons:
            isoform += genome_sequence[ex.start: ex.end]
            exons.append((total_len, ex.start, ex.end - ex.start))
            total_len += ex.end - ex.start
        known_transcriptome[gene][iso] = [isoform, exons]
print("time to find the known transcriptome: " + str(t + time()))

time to find the known transcriptome: 0.0009839534759521484


In [40]:
t = -time()
unknown_transcriptome = {}
for gene in unknown_genes:
    unknown_transcriptome[gene] = {}
    for iso in unknown_genes[gene].isoforms:
        isoform, exons, total_len = '', [], 0
        for ex in iso.exons:
            isoform += genome_sequence[ex.start: ex.end]
            exons.append((total_len, ex.start, ex.end - ex.start))
            total_len += ex.end - ex.start
        unknown_transcriptome[gene][iso] = [isoform, exons]
print("time to find the UNknown transcriptome: " + str(t + time()))

time to find the UNknown transcriptome: 0.00026702880859375


In [21]:
num_isoforms = 0
for gene in known_transcriptome:
    num_isoforms += len(known_transcriptome[gene])
print('number of known isoforms:', num_isoforms)

number of known isoforms: 127


In [22]:
num_isoforms = 0
for gene in unknown_transcriptome:
    num_isoforms += len(unknown_transcriptome[gene])
print('number of UNknown isoforms:', num_isoforms)

number of UNknown isoforms: 9


In [63]:
def find_alignment(read_len, align_start, exons):
    def find_start_location(lo, hi):
        mid = (lo + hi) // 2
        if exons[mid][0] <= align_start < exons[mid][0] + exons[mid][2]:
            return mid
        elif exons[mid][0] + exons[mid][2] <= align_start:
            return find_start_location(mid+1, hi)
        else:
            return find_start_location(lo, mid-1)
    idx = find_start_location(0, len(exons))
    algn = [(0, exons[idx][1] + (align_start - exons[idx][0]),
             read_len if read_len + (align_start - exons[idx][0]) <= exons[idx][2]
             else exons[idx][2] - (align_start - exons[idx][0]))]
    read_len -= exons[idx][2] - (align_start - exons[idx][0])
    while read_len > 0:
        idx += 1
        algn.append((0, exons[idx][1] + (align_start - exons[idx][0]),
                     read_len if read_len + (align_start - exons[idx][0]) <= exons[idx][2]
                     else exons[idx][2] - (align_start - exons[idx][0])))
        read_len -= exons[idx][2] - (align_start - exons[idx][0])
    return algn

In [64]:
def align_to_isoform(read, isoform, exons):
    match = (-1, float("inf"))
    for i in range(len(isoform) - len(read) + 1):
        j, mismatches = 0, 0
        while j < len(read):
            if isoform[i+j] != read[j]:
                mismatches += 1
            if mismatches > 6:
                break
            j += 1
        if j == len(read) and mismatches < match[1]:
            match = (i, mismatches)
    return match if match[0] == -1 else (find_alignment(len(read), match[0], exons), match[1])

In [65]:
def align_to_transcriptome(read, transcriptome):
    match = (-1, float("inf"))
    for gene in transcriptome:
        for iso in transcriptome[gene]:
            alignment, mismatches = align_to_isoform(read, transcriptome[gene][iso][0], transcriptome[gene][iso][1])
            if alignment != -1 and mismatches <= match[1]:
                match = (alignment, mismatches)
    return match

In [59]:
reads[0][1]

'ATTACTCTTGGGAATGAAATCCTATCTATATAAGCTGTGGTTTGAAATCC'

In [60]:
rand_iso

NameError: name 'rand_iso' is not defined

In [66]:
t = -time()
# align_to_isoform(reads[166][1], reads[166][1])
align_to_transcriptome(reads[150][1], known_transcriptome)

([(0, 6759550, 50)], 2)

In [126]:
unable_to_match = []
for i in range(len(reads)):
    start, mismatches = align_to_transcriptome(reads[i][1], known_transcriptome)
    if start == -1:
        unable_to_match.append(i)
        print(str(i) + ' not matched!')

0 not matched!
1 not matched!
2 not matched!
3 not matched!
4 not matched!
5 not matched!
6 not matched!
7 not matched!
8 not matched!
9 not matched!
10 not matched!
11 not matched!
12 not matched!
13 not matched!
14 not matched!
15 not matched!
16 not matched!
17 not matched!
18 not matched!
19 not matched!
20 not matched!
21 not matched!
22 not matched!
23 not matched!
24 not matched!
25 not matched!
26 not matched!
27 not matched!
28 not matched!
29 not matched!
30 not matched!
31 not matched!
32 not matched!
33 not matched!
34 not matched!
35 not matched!
36 not matched!
37 not matched!
38 not matched!
39 not matched!
40 not matched!
41 not matched!
42 not matched!
43 not matched!
44 not matched!
45 not matched!
46 not matched!
47 not matched!
48 not matched!
49 not matched!
50 not matched!
51 not matched!
52 not matched!
53 not matched!
54 not matched!
55 not matched!
56 not matched!
57 not matched!
58 not matched!
59 not matched!
60 not matched!
61 not matched!
62 not matched!
63

723 not matched!
724 not matched!
725 not matched!
726 not matched!
727 not matched!
728 not matched!
729 not matched!
730 not matched!
731 not matched!
732 not matched!
733 not matched!
734 not matched!
735 not matched!
736 not matched!
737 not matched!
738 not matched!
739 not matched!
740 not matched!
741 not matched!
742 not matched!
743 not matched!
744 not matched!
745 not matched!
746 not matched!
747 not matched!
748 not matched!
749 not matched!
750 not matched!
751 not matched!
752 not matched!
753 not matched!
754 not matched!
755 not matched!
756 not matched!
757 not matched!
758 not matched!
759 not matched!
760 not matched!
761 not matched!
762 not matched!
763 not matched!
764 not matched!
765 not matched!
766 not matched!
767 not matched!
768 not matched!
769 not matched!
770 not matched!
771 not matched!
772 not matched!
773 not matched!
774 not matched!
775 not matched!
776 not matched!
777 not matched!
778 not matched!
779 not matched!
780 not matched!
781 not matche

1230 not matched!
1231 not matched!
1232 not matched!
1233 not matched!
1234 not matched!
1235 not matched!
1236 not matched!
1237 not matched!
1238 not matched!
1239 not matched!
1240 not matched!
1241 not matched!
1242 not matched!
1243 not matched!
1244 not matched!
1245 not matched!
1246 not matched!
1247 not matched!
1248 not matched!
1249 not matched!
1250 not matched!
1251 not matched!
1252 not matched!
1253 not matched!
1254 not matched!
1255 not matched!
1256 not matched!
1257 not matched!
1258 not matched!
1259 not matched!
1260 not matched!
1261 not matched!
1262 not matched!
1263 not matched!
1264 not matched!
1265 not matched!
1266 not matched!
1267 not matched!
1268 not matched!
1269 not matched!
1270 not matched!
1271 not matched!
1272 not matched!
1273 not matched!
1274 not matched!
1275 not matched!
1276 not matched!
1277 not matched!
1278 not matched!
1279 not matched!
1280 not matched!
1281 not matched!
1282 not matched!
1283 not matched!
1284 not matched!
1285 not m

In [99]:
still_unable_to_match = []
for i in range(len(unable_to_match)):
    start, mismatches = align_to_transcriptome(reads[i][1], unknown_transcriptome)
    if start != -1:
        print(str(i) + ' matched to an unknown gene!')
    else:
        still_unable_to_match.append(i)

8 matched to an unknown gene!


KeyboardInterrupt: 